In [125]:
import os
import json
import tiktoken
from dotenv import load_dotenv
from pprint import pprint
from langchain_community.document_loaders import DirectoryLoader
from langchain.prompts import PromptTemplate
from langchain_anthropic import ChatAnthropic

In [2]:
load_dotenv()

encoding = tiktoken.get_encoding("cl100k_base")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

In [3]:
loader = DirectoryLoader("../data/demo", glob="*.txt")
documents = loader.load()

for num, doc in enumerate(documents):
    print(
        f"index: {num}, file: {doc.metadata['source'][-15:]}, length: {len(doc.page_content)}, tokens: {len(encoding.encode(doc.page_content))}.")

index: 0, file: interview_2.txt, length: 33951, tokens: 15518.
index: 1, file: interview_1.txt, length: 36025, tokens: 16647.


In [4]:
interview_1 = documents[0].page_content
interview_2 = documents[1].page_content

In [215]:
llm_haiku = ChatAnthropic(api_key=ANTHROPIC_API_KEY, model="claude-3-haiku-20240307", max_tokens=4096, temperature=0.0)
llm_haiku_t = ChatAnthropic(api_key=ANTHROPIC_API_KEY, model="claude-3-haiku-20240307", max_tokens=4096, temperature=0.5)

In [78]:
def asks_llm(llm: None, raw_data: list[str] = None, text_template: str = None, input_variables: list[str] = None):
    """
    
    :param llm: 
    :param raw_interview: 
    :param text_template: 
    :param input_variables: 
    :param input_chain: 
    :return: 
    """

    upd = "\nAttention, this is important! Always send the full text of the response in Russian only. Thanks!"

    prompt = PromptTemplate(input_variables=input_variables, template=text_template + upd)
    chain = prompt | llm

    input_chain = {}
    for var, data in zip(input_variables, raw_data):
        input_chain[var] = data

    answer = chain.invoke(input_chain)
    return answer

In [105]:
def extract_llm_response(raw_answer):
    metadata = {
        "model": raw_answer.response_metadata["model"],
        "input_tokens": raw_answer.response_metadata["usage"]["input_tokens"],
        "output_tokens": raw_answer.response_metadata["usage"]["output_tokens"]
    }

    content = json.loads(raw_answer.content)

    return metadata, content

In [120]:
answer_1 = asks_llm(
    llm=llm_haiku, raw_data=[interview_1],
    text_template="""
        Пожалуйста, внимательно прочтите предоставленное вам интервью и следуйте приведенным ниже пошаговым инструкциям: <interview>{raw_interview}</interview>

        Пошаговые инструкции:
        1. Используйте только информацию, предоставленную вам в интервью, не придумывайте ничего самостоятельно.
        2. Выделите основные разделы или фазы интервью. Например: введение, общие вопросы, основные темы, заключительные замечания.
        3. Для каждого выделенного основного раздела приведите не более пяти цитат из вопросов в этом разделе. Важно, чтобы цитаты были короткими, но описывали этот раздел как можно полнее. Используйте только цитаты из вопросов интервьюера. Если в цитате встречается имя интервьюера или респондента, то удалите это имя.
        4 Верните окончательный ответ в формате json, где ключом будет название основного раздела интервью, а значениями список всех выбранных вами цитат. 
        """,
    input_variables=["raw_interview"],
)

In [121]:
answer_1_metadata, answer_1_content = extract_llm_response(answer_1)

In [171]:
answer_2 = asks_llm(
    llm=llm_haiku, raw_data=[interview_1],
    text_template="""
        Пожалуйста, внимательно прочтите предоставленное вам интервью и следуйте приведенным ниже пошаговым инструкциям: <interview>{raw_interview}</interview>

        Пошаговые инструкции:
        1. Используйте только информацию, предоставленную вам в интервью, не придумывайте ничего самостоятельно.
        2. Выделите ключевые фразы, словосочетания или предложения, отражающие важные идеи, концепции или опыт. 
        3. Присвоите получившимся выделенным сегментам короткие ярлыки. Используя короткие описательные фразы или слова, которые обобщают основную идею.
        4. Верните окончательный ответ в формате json, где ключом будет короткий ярлык, а значениями будут описания этого кода-ярлыка.
        """,
    input_variables=["raw_interview"],
)

In [172]:
answer_2_metadata, answer_2_content = extract_llm_response(answer_2)

In [174]:
answer_2_codes = ", ".join(list(answer_2_content.keys()))

In [194]:
answer_3 = asks_llm(
    llm=llm_haiku, raw_data=[interview_1, answer_2_codes],
    text_template="""
        Пожалуйста, внимательно прочтите предоставленное вам интервью и следуйте приведенным ниже пошаговым инструкциям: <interview>{raw_interview}</interview>

        Пошаговые инструкции:
        1. Используйте только информацию, предоставленную вам в интервью, не придумывайте ничего самостоятельно.
        2. Просмотрите тематические коды <interview_code>{interview_code}</interview_code> и найдите среди них сходства или взаимосвязи.
        3. Сгруппируйте связанные тематические коды в более широкие категории или темы, которые охватывают основные темы, обсуждавшиеся в интервью.
        4. Определите любые подтемы в рамках каждой основной темы, чтобы обеспечить более детальное понимание тем.
        5. Верните окончательный ответ в формате json, где ключами будут основные темы, а значениями список подтем. 
        """,
    input_variables=["raw_interview", "interview_code"]
)

In [195]:
answer_3_metadata, answer_3_content = extract_llm_response(answer_3)

In [202]:
answer_4 = asks_llm(
    llm=llm_haiku, raw_data=[interview_1],
    text_template="""
        Пожалуйста, внимательно прочтите предоставленное вам интервью и следуйте приведенным ниже пошаговым инструкциям: <interview>{raw_interview}</interview>

        Пошаговые инструкции:
        1. Используйте только информацию, предоставленную вам в интервью, не придумывайте ничего самостоятельно.
        2. Составьте список всех конкретных проблем, упомянутых респондентами.
        2. Подсчитайте, сколько раз упоминалась каждая проблема. 
        3. Верните окончательный ответ в формате json, где ключами будут проблемы, а значениями количество упоминаний проблем в интервью. 
        """,
    input_variables=["raw_interview"]
)

In [204]:
answer_4_metadata, answer_4_content = extract_llm_response(answer_4)

In [207]:
answer_4_problems = ", ".join(list(answer_4_content.keys()))

In [209]:
answer_5 = asks_llm(
    llm=llm_haiku, raw_data=[interview_1, answer_4_problems],
    text_template="""
        Пожалуйста, внимательно прочтите предоставленное вам интервью и следуйте приведенным ниже пошаговым инструкциям: <interview>{raw_interview}</interview>

        Пошаговые инструкции:
        1. Используйте только информацию, предоставленную вам в интервью, не придумывайте ничего самостоятельно.
        2. Просмотрите список проблем <interview_problems>{interview_problems}</interview_problems> и найдите среди них сходства или взаимосвязи.
        2. Сгруппируйте связанные проблемы в более широкие типы проблем, которые охватывают различные упомянутые проблемы в интервью. 
        
        4. Верните окончательный ответ в формате json, где ключами будут широкие типы проблем, а значениями списки проблем входящих в них.
        """,
    input_variables=["raw_interview", "interview_problems"]
)

In [211]:
answer_5_metadata, answer_5_content = extract_llm_response(answer_5)

In [213]:
answer_5_group_problems = ", ".join(list(answer_5_content.keys()))

In [237]:
answer_6 = asks_llm(
    llm=llm_haiku_t, raw_data=[interview_1, answer_5_group_problems],
    text_template="""
        Пожалуйста, внимательно прочтите предоставленное вам интервью и категории проблем, которые были выявлены при первичном анализе данного интервью: <interview>{raw_interview}</interview> and <interview_problems>{interview_problems_categories}</interview_problems> 


        Пошаговые инструкции:
        1. Используйте только информацию, предоставленную вам в интервью и в выявленных категориях проблем, не придумывайте ничего самостоятельно.
        
        2. Проанализируйте темы и частоту возникновения проблем и их категории. Дайте ответ в форме вложенного json, верхнеуровневым ключом будет – "reflections", вложенными ключами будут категория проблем, а значениями будет список содержащий результаты проведенного анализа.  
        
        3. Проанализируйте, что результаты "reflections" говорят об опыте, взглядах или трудностях респондента. Дайте ответ в форме вложенного json, верхнеуровневым ключом будет – "results", вложенными ключами будут категория опыта, а значениями будет список содержащий результаты проведенного анализа.
        
        4. Обратите внимание на любые неожиданные закономерности, связи или контрасты в данных. Поделитесь этими наблюдениями. Дайте ответ в форме вложенного json, верхнеуровневым ключом будет – "unexpected", вложенными ключами будут категория контрастов, а значениями будет список содержащий результаты проведенного анализа.
        
        5. Разработайте предварительные объяснения или гипотезы о том, почему возникли определенные темы или проблемы и как они могут быть связаны друг с другом или с более широкими контекстуальными факторами. Дайте ответ в форме json, где будет только один ключ – "hypothesis", а значениями будет список содержащий результаты проведенного анализа.
        
        6. Рассмотрите альтернативные объяснения и контрпримеры, чтобы прояснить гипотезы сформированные в разделе "hypothesis". Дайте ответ в форме json, где будет только один ключ – "alternatives", а значениями будет список содержащий результаты проведенного анализа.
        
        7. Укажите области, в которых могут потребоваться дополнительные исследования или анализ для подтверждения ваших идей из раздела "alternatives". Дайте ответ в форме json, где будет только один ключ – "additional", а значениями будет список содержащий результаты проведенного анализа.
        """, 
    input_variables=["raw_interview", "interview_problems_categories"]
)

In [239]:
answer_6_metadata, answer_6_content = extract_llm_response(answer_6)

In [249]:
results = {
    "step_1": answer_1_content,
    "step_2": answer_2_content,
    "step_3": answer_3_content,
    "step_4": answer_4_content,
    "step_5": answer_5_content,
    "step_6": answer_6_content,
}

In [255]:
with open("../data/demo/results_one_interview.json", "w", encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False)